In [1]:
import tensorflow as tf
from skimage import io
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import Model
import os
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
print(tf.__version__)

2.4.1


In [3]:
model_loaded = tf.keras.models.load_model('feature_extraction.h5')

In [4]:
model_loaded.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [5]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32
img_width = 224
img_height = 224

layer_name = 'dense_6'
intermediate_layer_model = Model(inputs=model_loaded.input, outputs=model_loaded.get_layer(layer_name).output)

In [6]:
def extract_features(directory, sample_count, intermediate_layer_model):
    features = np.zeros(shape=(sample_count, 256))  # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count))
    # Preprocess data
    generator = datagen.flow_from_directory(directory,
                                            target_size=(img_width,img_height),
                                            batch_size = batch_size,
                                            class_mode='binary')
    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = intermediate_layer_model.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

In [7]:
train_dir = "Dataset_anti_spoofing/train/"
train_features, train_labels = extract_features(train_dir, 6000, intermediate_layer_model)

Found 6000 images belonging to 2 classes.


In [8]:
test_dir = "Dataset_anti_spoofing/test/"
test_features, test_labels = extract_features(test_dir, 2000, intermediate_layer_model)

Found 2000 images belonging to 2 classes.


In [9]:
np.save("Dataset_anti_spoofing/train/features_tf_train.npy", train_features)
np.save("Dataset_anti_spoofing/train/labels_tf_train.npy", train_labels)
np.save("Dataset_anti_spoofing/test/features_tf_test.npy", test_features)
np.save("Dataset_anti_spoofing/test/labels_tf_test.npy", test_labels)

In [10]:
X_train = np.load("Dataset_anti_spoofing/train_old/tf_features_train.npy")
y_train = np.load("Dataset_anti_spoofing/train_old/tf_labels_train.npy")
X_test = np.load("Dataset_anti_spoofing/test_old/tf_features_test.npy")
y_test = np.load("Dataset_anti_spoofing/test_old/tf_labels_test.npy")